<a href="https://colab.research.google.com/github/vitaliy-sharandin/data_science_projects/blob/master/portfolio/nlp/fine-tuned-llm/psy_ai_wandb_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Psy AI
Psy AI is a Llama3-8b model instruction fine-tuned on depression dataset and is meant to help improve mental well-being.

In future iterations it is meant to be trained on multiple philosophical and psychological datasets in order to provide multifaceted answers to complex mental health issues.



# Datasets

Philosophy datasets (* for future training)
* https://www.kaggle.com/datasets/christopherlemke/philosophical-texts
* https://www.workwithdata.com/object/philosophy-science-complete-a-text-on-traditional-problems-schools-thought-book-by-edwin-h-c-hung-0000
* https://www.kaggle.com/datasets/christopherlemke/philosophy-authors-writings-german
* https://www.workwithdata.com/object/philosophical-inquiries-an-introduction-to-problems-philosophy-book-by-nicholas-rescher-0000
* https://www.workwithdata.com/object/roman-stoicism-book-by-edward-vernon-arnold-1857
* https://www.workwithdata.com/object/wisdom-energy-basic-buddhist-teachings-book-by-thubten-yeshe-1935

Psychology and mental health datasets

* Kaggle Depression data for chatbot https://www.kaggle.com/datasets/nupurgopali/depression-data-for-chatbot
* Kaggle Psychometrics dataset https://www.kaggle.com/discussions/general/304994
* Psychometric tests dataset https://ieee-dataport.org/documents/psychometric-tests-dataset
* Psychometric NLP https://paperswithcode.com/dataset/psychometric-nlp
* Reddit mental health dataset https://zenodo.org/record/3941387
* Reddit mental disorders identification https://www.kaggle.com/datasets/kamaruladha/mental-disorders-identification-reddit-nlp
* Kaggle Mental Health Conversational Data https://www.kaggle.com/datasets/elvis23/mental-health-conversational-data
* Kaggle Mental Health FAQ for Chatbot https://www.kaggle.com/narendrageek/mental-health-faq-for-chatbot/code
* A human consciousness questionnaire dataset https://data.mendeley.com/datasets/69p62ksdh6
* paperswithcode Self-reported Mental Health Diagnoses https://paperswithcode.com/dataset/smhd
* paperswithcode Mental Health Summarization Dataset https://paperswithcode.com/dataset/mentsum
* HuggingFace psychology dataset https://huggingface.co/datasets/samhog/psychology-10k


In [1]:
!pip install -U -q gradio
!pip install -U -q datasets
!pip install -U -q bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -U -q trl

!pip install -U -q evaluate
!pip install -U -q rouge_score
!pip install -U -q optuna
!pip install -U -q wandb

In [2]:
import os
from datasets import load_dataset
import json
import yaml
import gradio as gr
import torch
import transformers
from transformers import GenerationConfig, Trainer, TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import Dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel, AutoPeftModelForCausalLM
import numpy as np
from evaluate import load
import optuna
import wandb
import datetime

import warnings
warnings.filterwarnings('ignore')

# Dataset instruction transformation

Depression dataset with 51 q&a entries was taken for mvp to see if model is indeed training.

In [3]:
depression_dataset = load_dataset("vitaliy-sharandin/depression-instruct")

First, we modify our dataset to Alpaca format and create two datasets. One - for testing and evaluation and second one for inference, where responses are not available in formatted prompt.

In [4]:
def formatting_func_train(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n"
      f"{example['response']}")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response:\n"
      f"{example['response']}")

  return {"text" : input_prompt}

def formatting_func_test(example):
  if example.get("context", "") != "":
      input_prompt = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Input: \n"
      f"{example['context']}\n\n"
      f"### Response: \n")

  else:
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{example['instruction']}\n\n"
      f"### Response: \n")

  return {"text" : input_prompt}

In [5]:
formatted_depression_dataset_train = depression_dataset['train'].map(formatting_func_train)
formatted_depression_dataset_test = depression_dataset['train'].map(formatting_func_test)

# Model load

We are loading our Llama2 model in 4bit quantized form as well as applying Lora for peft training.

In [6]:
model_name = "NousResearch/Meta-Llama-3-8B-Instruct"

def get_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  tokenizer.pad_token = tokenizer.eos_token
  tokenizer.padding_side = "right"
  return tokenizer

def get_model():


  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )

  qlora_config = LoraConfig(lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM")

  base_training_model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config=bnb_config,
      device_map = {"": 0}
  )

  base_training_model.gradient_checkpointing_enable()

  base_training_model = prepare_model_for_kbit_training(base_training_model)
  base_training_model = get_peft_model(base_training_model, qlora_config)
  return base_training_model.to('cuda')

base_training_model = get_model()
tokenizer = get_tokenizer()

torch.manual_seed(42)
print(base_training_model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=F

# Model instruction fine-tuning

Here we are defining inference function which returns bleu, rouge and f1 metrics after comparison of predicted and reference responses. My tests have shown that standard trainer `compute_metrics` method is quite inefficient and is not quite suitable for instruction fine-tuning during manual observations or generated results.

In [7]:
entity="vitaliy-sharandin-org"
project_name="Psy-AI"
os.environ["WANDB_LOG_MODEL"]="true"

model_trained_checkpoint = 'llama3-trained'
registry_model_name = "psy-ai-model"

def bleu_rouge_f1_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=-1)

  labels = [[idx for idx in label if idx != -100] for label in labels]
  predictions = [[idx for idx in prediction if idx != -100] for prediction in predictions]

  decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  bleu = load("bleu")
  bleu_results = bleu.compute(predictions=decoded_predictions, references=decoded_labels)

  rouge = load('rouge')
  rouge_results = rouge.compute(predictions=decoded_predictions, references=decoded_labels)

  f1 = 2 * (bleu_results['bleu'] * rouge_results['rouge1']) / (bleu_results['bleu'] + rouge_results['rouge1'])

  scores = {
        "bleu": bleu_results["bleu"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "f1": f1
    }

  return scores

def fine_tune(model, tokenizer, train_dataset, eval_dataset, metrics_func, only_evaluate=False):

  supervised_finetuning_trainer = SFTTrainer(model=model,
                                            train_dataset=train_dataset,
                                            eval_dataset=eval_dataset,
                                            args=TrainingArguments(
                                                output_dir="./training_results",
                                                report_to="wandb",
                                                per_device_train_batch_size=8,
                                                per_device_eval_batch_size=1,
                                                gradient_accumulation_steps=2,
                                                gradient_checkpointing=True,
                                                ddp_find_unused_parameters=False,
                                                optim="paged_adamw_8bit",
                                                save_steps=1000,
                                                logging_steps=30,
                                                learning_rate=2e-4,
                                                weight_decay=0.001,
                                                fp16=False,
                                                bf16=False,
                                                max_grad_norm=0.3,
                                                max_steps=-1,
                                                warmup_ratio=0.3,
                                                group_by_length=True,
                                                lr_scheduler_type="constant"
                                            ),
                                            dataset_text_field="text",
                                            max_seq_length=2048,
                                            compute_metrics=metrics_func,
                                            data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False))

  if only_evaluate:
    return supervised_finetuning_trainer.evaluate()

  else:
    with wandb.init(entity=entity, project=project_name) as run:
      supervised_finetuning_trainer.train()
      supervised_finetuning_trainer.model.save_pretrained(model_trained_checkpoint)
      tokenizer.save_pretrained(model_trained_checkpoint)
      supervised_finetuning_trainer.evaluate()

      run.link_model(path=model_trained_checkpoint, registered_model_name=registry_model_name)

      wandb.finish()

## Evaluating base model

In [8]:
params = {
    "model": base_training_model,
    "tokenizer": tokenizer,
    "train_dataset": formatted_depression_dataset_train,
    "eval_dataset": formatted_depression_dataset_train,
    "metrics_func": bleu_rouge_f1_metrics,
    "only_evaluate": True
}

fine_tune(**params)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


wandb: Currently logged in as: vitaliy-sharandin (vitaliy-sharandin-org). Use `wandb login --relogin` to force relogin


{'eval_loss': 3.269576072692871,
 'eval_bleu': 0.09570116650916786,
 'eval_rouge1': 0.5119052674461926,
 'eval_rouge2': 0.13663171863412898,
 'eval_rougeL': 0.3936583889562995,
 'eval_f1': 0.16125547228944379,
 'eval_runtime': 56.781,
 'eval_samples_per_second': 0.898,
 'eval_steps_per_second': 0.898}

Evaluation metrics are quite low before training.

Let's train the model and see how metrics and responses change.

## Fine-tuning base model and registering it to WandB model registry


In [9]:
params = {
    "model": base_training_model,
    "tokenizer": tokenizer,
    "train_dataset": formatted_depression_dataset_train,
    "eval_dataset": formatted_depression_dataset_train,
    "metrics_func": bleu_rouge_f1_metrics
}

fine_tune(**params)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


eval/bleu,▁
eval/f1,▁
eval/loss,▁
eval/rouge1,▁
eval/rouge2,▁
eval/rougeL,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/global_step,▁
eval/bleu,0.0957


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


wandb: Adding directory to artifact (./llama3-trained)... Done. 0.7s


eval/bleu,▁
eval/f1,▁
eval/loss,▁
eval/rouge1,▁
eval/rouge2,▁
eval/rougeL,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁
train/global_step,▁▁


We can see that after fine-tuning evaluation metrics have improved to some extent, so our training was successful.

## Loading model from Wandb model registry

Now we can load the model from the model registry and use it with `AutoModelForCausalLM` just as we would with any other model

In [10]:
run = wandb.init()
downloaded_model_path = run.use_model(name=f"{entity}/model-registry/{registry_model_name}:latest")

wandb: Downloading large artifact psy-ai-model:latest, 112.74MB. 6 files... 
wandb:   6 of 6 files downloaded.  
Done. 0:0:0.4


# Gradio demo in Hugging Face Spaces

https://huggingface.co/spaces/vitaliy-sharandin/WiseAI

## Local launch
Requirements:
* Choose free T4 GPU in Colab environment settings
* Run `pip install` and `imports` cells at the beginning before launching Gradio.

In [ ]:
run = wandb.init()
downloaded_model_path = run.use_model(name=f"{entity}/model-registry/{registry_model_name}:latest")

model = AutoModelForCausalLM.from_pretrained(
      downloaded_model_path,
      load_in_8bit=True,
      device_map = {"": 0}
  )
tokenizer = AutoTokenizer.from_pretrained(downloaded_model_path)

pipe = pipeline('text-generation', model=model,tokenizer=tokenizer)

def generate_text(instruction, input):
  if not instruction.strip():
    return str('The instruction field is required.')

  if instruction.strip() and input.strip():
    input_prompt = (f"Below is an instruction that describes a task. "
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{instruction}\n\n"
      "### Input:\n"
      f"{input}\n\n"
      f"### Response: \n")
  else :
    input_prompt = (f"Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n"
        f"{instruction}\n\n"
        f"### Response: \n")
  result = pipe(input_prompt, max_length=200, top_p=0.9, temperature=0.9, num_return_sequences=1, return_full_text=False)[0]['generated_text']
  return result[:str(result).find("###")]

iface = gr.Interface(fn=generate_text, inputs=[gr.Textbox(label="Instruction"),
                                               gr.Textbox(label="Additional Input")],
                     outputs=gr.Textbox(label="Response"))
iface.launch()